In [16]:
import pandas as pd
import numpy as np
from ete3 import NCBITaxa
import tempfile
import subprocess
import os
import io
import re
import time
import json
import math
ncbi = NCBITaxa()

In [18]:
gitdir = "/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/Combined_SENZOR_results/"
identity_qcov_threshold = 0.9

In [20]:
## Load metadata
metadata = pd.read_csv(gitdir + "Metada_annotated.csv")
idseq_data = pd.read_csv(gitdir + "correct_Idseq_metatda.csv", header=0)
idseq_data = idseq_data[~idseq_data['sample'].str.contains('Placeholder')]

In [22]:
metadata

,NewIDseqName,oldIDseqName,ska_genus,ska_species,collected_by,sample_type,nucleotide_type,collection_lat,collection_long,collection_date
0,AIAMA_GOT005_OS_S43_673459,AIAMA_GOT005_OS_S43,Caprine,Goat,Ebonyi,swab,RNA,6.2649,8.0137,01/10/2024
1,AIAMA_GOT006_RS_S44_673460,AIAMA_GOT006_RS_S44,Caprine,Goat,Ebonyi,swab,RNA,6.2649,8.0137,01/10/2024
2,AIAMA_GOT007_OS_S45_673461,AIAMA_GOT007_OS_S45,Caprine,Goat,Ebonyi,swab,RNA,6.2649,8.0137,2024-08
3,AIAMA_LIZ002_OS_S50_673462,AIAMA_LIZ002_OS_S50,Saurian,Lizard,Ebonyi,swab,RNA,6.2649,8.0137,2024-08
4,AIAMA_PIG003_RS_S51_673463,AIAMA_PIG003_RS_S51,Swine,Pig,Ebonyi,swab,RNA,6.2649,8.0137,10/08/2024
...,...,...,...,...,...,...,...,...,...,...
237,SEZOOWOGDOG011_S8_1_657761,SEZOOWOGDOG011,Canine,Dog,Ondo,swap,RNA,6.9149,5.1470,23/10/2021
238,SEZOOWOGDOG019_S9_1_657762,SEZOOWOGDOG019,Canine,Dog,Ondo,swap,RNA,6.9149,5.1470,02/02/2022
239,SSP_ERCC_S72_670948,SSP_ERCC_S72,control,water,NaN,NaN,NaN,6.9149,5.1470,02/02/2022
240,SSP_ERCC_S83_666426,SSP_ERCC_S83,water,water,NaN,NaN,NaN,6.9149,5.1470,23/10/2021


In [24]:
## Load contig data
contig_calls_all = pd.read_csv(gitdir + "contig_calls.tsv", sep="\t", header=0)
contig_calls_decontam = pd.read_csv(gitdir + "contig_calls_decontam.tsv", sep="\t", header=0)

/var/folders/xt/vz8fznds4hdbfcxyds95mtp00000gp/T/ipykernel_94804/1586704730.py:2: DtypeWarning: Columns (4,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  contig_calls_all = pd.read_csv(gitdir + "contig_calls.tsv", sep="\t", header=0)
/var/folders/xt/vz8fznds4hdbfcxyds95mtp00000gp/T/ipykernel_94804/1586704730.py:3: DtypeWarning: Columns (4,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  contig_calls_decontam = pd.read_csv(gitdir + "contig_calls_decontam.tsv", sep="\t", header=0)


In [25]:
print(metadata.head())

                 NewIDseqName         oldIDseqName ska_genus ska_species  \
0  AIAMA_GOT005_OS_S43_673459  AIAMA_GOT005_OS_S43   Caprine        Goat   
1  AIAMA_GOT006_RS_S44_673460  AIAMA_GOT006_RS_S44   Caprine        Goat   
2  AIAMA_GOT007_OS_S45_673461  AIAMA_GOT007_OS_S45   Caprine        Goat   
3  AIAMA_LIZ002_OS_S50_673462  AIAMA_LIZ002_OS_S50   Saurian      Lizard   
4  AIAMA_PIG003_RS_S51_673463  AIAMA_PIG003_RS_S51     Swine         Pig   

  collected_by sample_type nucleotide_type  collection_lat  collection_long  \
0      Ebonyi         swab             RNA          6.2649           8.0137   
1      Ebonyi         swab             RNA          6.2649           8.0137   
2      Ebonyi         swab             RNA          6.2649           8.0137   
3      Ebonyi         swab             RNA          6.2649           8.0137   
4      Ebonyi         swab             RNA          6.2649           8.0137   

  collection_date  
0      01/10/2024  
1      01/10/2024  
2       

In [32]:
import pandas as pd

# Check which columns are present (just for sanity)
print("Columns in metadata:", metadata.columns.tolist())

# Rename the appropriate column to 'sample'
metadata = metadata.rename(columns={"NewIDseqName": "sample"})

# Define which columns you want from metadata
metadata_cols = ["sample", "ska_genus", "ska_species", "collected_by"]
idseq_cols = ["sample", "nonhost_reads", "total_reads"]

# Calculate reads removed during decontamination and hexapoda subtraction
contam_counts = (
    pd.DataFrame(
        contig_calls_all.groupby("sample")["read_count"].sum()
        - contig_calls_decontam.groupby("sample")["read_count"].sum()
    )
    .rename(columns={"read_count": "contaminating_and_hex_reads"})
)

# Sum reads to hexapoda contigs by sample
contigs_hex = contig_calls_all[
    (contig_calls_all["chordata"] == True) & (contig_calls_all["curated"] == False)
]
hex_counts = (
    contigs_hex
    .groupby("sample")["read_count"]
    .sum()
    .reset_index()
    .rename(columns={"read_count": "hex_read_count"})
)

# Build partial DataFrames, setting "sample" as the index
metadata_part = (
    metadata[metadata_cols]
    .set_index("sample")
)

idseq_part = (
    idseq_data[idseq_cols]
    .rename(columns={"nonhost_reads": "idseq_nonhost_reads"})
    .set_index("sample")
)

hex_part = hex_counts.set_index("sample")
contam_part = contam_counts  # Also indexed by sample

# Concatenate the pieces
metadata = pd.concat(
    [
        metadata_part,
        idseq_part,
        hex_part,
        contam_part
    ],
    axis=1,
    sort=True
)

# Fill NAs without inplace to avoid chained assignment warnings
metadata["hex_read_count"] = metadata["hex_read_count"].fillna(0)
metadata["contaminating_and_hex_reads"] = metadata["contaminating_and_hex_reads"].fillna(0)

# Calculate final nonhost reads
metadata["nonhost_reads"] = (
    metadata["idseq_nonhost_reads"] - metadata["contaminating_and_hex_reads"]
)

# Reset index so "sample" becomes a column again.
metadata.reset_index(inplace=True)

# Reorder columns if desired
desired_cols = [
    "sample",
    "ska_genus",
    "ska_species",
    "collected_by",
    "idseq_nonhost_reads",
    "hex_read_count",
    "contaminating_and_hex_reads",
    "nonhost_reads",
    "total_reads"
]
metadata = metadata.reindex(columns=[c for c in desired_cols if c in metadata.columns])

print(metadata.head())


Columns in metadata: ['NewIDseqName', 'oldIDseqName', 'ska_genus', 'ska_species', 'collected_by', 'sample_type', 'nucleotide_type', 'collection_lat', 'collection_long', 'collection_date']
                       sample ska_genus ska_species collected_by  \
0  AIAMAAGR001_LIV_S46_662105       Rat      Rodent      Ebonyi    
1      AIAMACAT001_S69_657769    Feline         Cat       Ebonyi   
2      AIAMADOG001_S40_657732    Canine         Dog       Ebonyi   
3   AIAMADOG002_OS_S64_673467    Canine         Dog      Ebonyi    
4     AIAMAGOAT001_S48_656109   Caprine        Goat       Ebonyi   

   idseq_nonhost_reads  hex_read_count  contaminating_and_hex_reads  \
0             19926566         27340.0                      27399.0   
1              8894052           846.0                     657528.0   
2              1627364          7483.0                     398136.0   
3              2135538         10770.0                     327750.0   
4             21562762        392743.0          

In [34]:
# Write final nonhost reads to file
metadata[['sample','nonhost_reads']].to_csv("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/Combined_SENZOR_results/nonhost_reads_decontam_nohexapoda.tsv", 
                                            index=False, sep="\t")

In [36]:
# Update contig stats with nonhost reads
contig_stats = pd.merge(contig_calls_decontam, metadata, how="left", on="sample")
# Check no hexapoda remain
print( contig_stats[(contig_stats.chordata == True) & (contig_stats.curated == False)].shape[0] == 0 ) #shld be True
# Calculate read proportion and identity/query-coverage threshold
contig_stats['read_prop'] = contig_stats['read_count'] / contig_stats['nonhost_reads']
# Calculate identity/query-coverage threshold
contig_stats = contig_stats.assign(identity_qcov=(contig_stats["identity"]/100 
                                                  * contig_stats["align_length"]/contig_stats["contig_length"]
                                                 ).apply(lambda x: min(x, 1)))
contig_stats.loc[contig_stats['curated'] == True, 'identity_qcov'] = np.nan

True


In [38]:
contig_stats_viral = contig_stats[(contig_stats["taxon_group"]=="Viruses") & contig_stats["curated"]==True]

In [40]:
# Information about Baltimore classification of virus family groups
viral_family_groups = pd.read_csv("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/Combined_SENZOR_results/virus_family_groups.csv", header=0)
viral_family_groups = viral_family_groups.loc[:, ~viral_family_groups.columns.str.startswith('Unnamed')]
new_row = pd.DataFrame([{
    "family":"Nairoviridae",
    "baltimore_group":"V",
    "genome_description":"",
    "number_of_segments_conventional":""
}])

viral_family_groups = pd.concat([viral_family_groups, new_row], ignore_index=True)


# Convert virus json into data frame
with open ("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/Combined_SENZOR_results/virus_RNA.json", "r") as f:
    viral_json = pd.DataFrame(json.load(f)).T
    viral_json['poly_group'] = viral_json.index

# Check which viral families not represented
print(set(viral_json.family.unique()) - set (viral_family_groups.family.unique())) #should be empty

# Merge viral info with contig stats
viral_info = viral_json.merge(viral_family_groups,on='family')
viral_info.provisional_name.fillna(viral_info.name, inplace=True)
viral_info.rename(columns={'provisional_name':'sci_name'},inplace=True)
contigs_viral = contig_stats_viral.merge(viral_info[['poly_group','family','baltimore_group',
                                                     'genome_description','number_of_segments_conventional',
                                                     'sci_name']].
                                              astype({'poly_group': 'float64'}),
                                              on='poly_group', how = 'left')
contigs_viral = contigs_viral.assign(group="Virus")



{'Demerecviridae', 'Totiviridae', 'Arenaviridae ', 'unclassified', 'Schitoviridae', 'Zierdtviridae', 'Calicivirdae', 'Paramyxoviridae', 'Reoviridae', 'Bromoviridae', 'Solinviviridae'}


/var/folders/xt/vz8fznds4hdbfcxyds95mtp00000gp/T/ipykernel_94804/2881725343.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  viral_info.provisional_name.fillna(viral_info.name, inplace=True)


In [42]:
column_names = contigs_viral.columns 
column_names

Index(['sample', 'contig_name', 'contig_length', 'read_count', 'nr',
       'nt_or_nr', 'bitscore', 'align_length', 'taxon_group_blast', 'tax_id',
       'identity', 'sci_name_x', 'com_names', 'lca_taxid', 'lca_taxid.1',
       'lca_name', 'common_nr', 'phylum', 'family_x', 'chordata', 'blasted',
       'cluster', 'taxid_curated', 'poly_group', 'name', 'provisional_name',
       'segment', 'curated', 'rdrp', 'taxon_group', 'taxid', 'ska_genus',
       'ska_species', 'collected_by', 'idseq_nonhost_reads', 'hex_read_count',
       'contaminating_and_hex_reads', 'nonhost_reads', 'total_reads',
       'read_prop', 'identity_qcov', 'family_y', 'baltimore_group',
       'genome_description', 'number_of_segments_conventional', 'sci_name_y',
       'group'],
      dtype='object')

In [44]:
def get_rows_taxid(df, taxid, taxid_colname="taxid"):
    if (not isinstance(df, pd.DataFrame)):
        if isinstance(taxid, str):
            taxid = ncbi.get_name_translator([taxid])[taxid][0]
        return (taxid in ncbi.get_lineage(df))
    outdf = df[df[taxid_colname].apply(get_rows_taxid, taxid=taxid)]
    return (outdf) 

def group_at_higher_tax(df, taxonomic_group, family_name, taxid_colname="taxid", family_colname="family"):
    groups = {}
    for x in taxonomic_group:
        if isinstance(x, str):
            groups[x] = ncbi.get_name_translator([x])[x][0]
        else:
            groups[ncbi.get_taxid_translator(x)[x]] = x
    family_assignments = {}
    for x in df["taxid"].unique():
        lin = ncbi.get_lineage(x)
        family_assignments[x] = family_name
        for key, taxid_x in groups.items():
            if taxid_x in lin:
                family_assignments[x] = key
                break
    df[family_colname] = df["taxid"].apply(lambda x: family_assignments[x])
    return (df)

def get_summary_table (df, colnames, metric):
    df = df.groupby(colnames)[metric].sum().reset_index()
    if not isinstance(metric, list):
        metric = [metric]
    sort_order = colnames+metric
    sort_order.remove("sample")
    return (df.sort_values(by=sort_order, ascending=False))

In [46]:
# Subset to contigs that are not viral, and are strong NCBI hits based on identity/qcov threshold
contig_stats_nonviral = contig_stats[ (contig_stats["taxon_group"]!="Viruses") &
                                    (contig_stats["identity_qcov"]) ]

In [48]:
contig_stats_nonviral

,sample,contig_name,contig_length,read_count,nr,nt_or_nr,bitscore,align_length,taxon_group_blast,tax_id,...,ska_genus,ska_species,collected_by,idseq_nonhost_reads,hex_read_count,contaminating_and_hex_reads,nonhost_reads,total_reads,read_prop,identity_qcov
4,AIAMAAGR001_LIV_S46_662105,AIAMAAGR001_LIV_S46_662105_NODE_1801_length_40...,405,9,True,nr,114.0,92.0,Bacteria,2849492.0,...,Rat,Rodent,Ebonyi,19926566,27340.0,27399.0,19899167.0,34756400,4.522802e-07,0.130844
10,AIAMAAGR001_LIV_S46_662105,AIAMAAGR001_LIV_S46_662105_NODE_1_length_22617...,22617,2245,True,nr,542.0,1855.0,Metazoa,79923.0,...,Rat,Rodent,Ebonyi,19926566,27340.0,27399.0,19899167.0,34756400,1.128188e-04,0.029608
12,AIAMAAGR001_LIV_S46_662105,AIAMAAGR001_LIV_S46_662105_NODE_217_length_916...,916,62,True,nr,53.1,45.0,Metazoa,45882.0,...,Rat,Rodent,Ebonyi,19926566,27340.0,27399.0,19899167.0,34756400,3.115708e-06,0.027314
13,AIAMAAGR001_LIV_S46_662105,AIAMAAGR001_LIV_S46_662105_NODE_22396_length_2...,249,4,True,nr,101.0,76.0,Bacteria,2849492.0,...,Rat,Rodent,Ebonyi,19926566,27340.0,27399.0,19899167.0,34756400,2.010134e-07,0.208771
14,AIAMAAGR001_LIV_S46_662105,AIAMAAGR001_LIV_S46_662105_NODE_25_length_2425...,2425,182,True,nr,316.0,605.0,Metazoa,79923.0,...,Rat,Rodent,Ebonyi,19926566,27340.0,27399.0,19899167.0,34756400,9.146111e-06,0.115761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584470,SEZONOWOGR004_LIV_S50_666375,SEZONOWOGR004_LIV_S50_666375_NODE_5302_length_...,394,6,True,nr,83.6,108.0,Metazoa,36087.0,...,Rat,Rodent,Ondo,3014762,110603.0,110778.0,2903984.0,12859300,2.066127e-06,0.121706
584471,SEZONOWOGR004_LIV_S50_666375,SEZONOWOGR004_LIV_S50_666375_NODE_5325_length_...,393,19,True,nr,50.8,28.0,Metazoa,268475.0,...,Rat,Rodent,Ondo,3014762,110603.0,110778.0,2903984.0,12859300,6.542736e-06,0.058494
584472,SEZONOWOGR004_LIV_S50_666375,SEZONOWOGR004_LIV_S50_666375_NODE_5341_length_...,393,5,True,nr,78.6,43.0,Metazoa,68888.0,...,Rat,Rodent,Ondo,3014762,110603.0,110778.0,2903984.0,12859300,1.721773e-06,0.091580
584473,SEZONOWOGR004_LIV_S50_666375,SEZONOWOGR004_LIV_S50_666375_NODE_5350_length_...,393,4,True,nr,71.2,53.0,Metazoa,68888.0,...,Rat,Rodent,Ondo,3014762,110603.0,110778.0,2903984.0,12859300,1.377418e-06,0.089008


In [50]:
## WOLBACHIA
wolbachia_taxid = 2
wolbachia_contigs = get_rows_taxid(contig_stats_nonviral, wolbachia_taxid);
## METAZOANS
metazoan_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"]=="Metazoa"];
# In the 'family' column, add information about the higher taxonomic grouping for taxids
metazoan_contigs = group_at_higher_tax(metazoan_contigs, 
                                       taxonomic_group=["Nematoda"], 
                                       family_name="Other metazoa", taxid_colname="taxid", family_colname="family")
## CHORDATA
chordate_contigs = get_rows_taxid(contig_stats_nonviral, taxid="Chordata");
# In the 'family' column, add information about the higher taxonomic grouping for taxids
chordate_contigs = group_at_higher_tax(chordate_contigs, 
                                       taxonomic_group=["Pecora", "Aves", "Carnivora", "Rodentia", "Leporidae"], 
                                       family_name="Other chordates", taxid_colname="taxid", family_colname="family")
## OTHER EUKARYOTES
eukaryotic_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"]=="Eukaryota"];
# In the 'family' column, add information about the higher taxonomic grouping for taxids
eukaryotic_contigs = group_at_higher_tax(eukaryotic_contigs, 
                                         taxonomic_group=["Trypanosomatidae", "Apicomplexa", "Schistosomatidae", "Onchocercidae", "Microsporidia"], 
                                         family_name="Other eukaryotes", taxid_colname="taxid", family_colname="family")

/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 85948 was translated into 147567
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 2991674 was translated into 3383038
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 7604 was translated into 133437
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 2052317 was translated into 3107663
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiqu

In [51]:
# Combine nonviral
contigs_nonviral = pd.concat([wolbachia_contigs.assign(group="Bacteria").assign(family="Bacteria"), 
                            metazoan_contigs.assign(group="Metazoa"),
                            chordate_contigs.assign(group="Chordata"),
                            eukaryotic_contigs.assign(group="Other Eukaryotes")], sort=False)
# Convert taxids to human_readable
contigs_nonviral = contigs_nonviral.assign(sci_name=contigs_nonviral["taxid"].
                                           apply(lambda x: ncbi.get_taxid_translator([x])[x]))
contigs_nonviral.loc[contigs_nonviral['group']=='Bacteria', ['sci_name']] = 'Bacteria'

In [52]:
contigs_out = pd.concat([contigs_nonviral, contigs_viral], sort=False)
# Count total number of contigs in each broad group
numbers = contigs_out.groupby("group")["group"].count()
print(numbers)

group
Bacteria            478447
Metazoa               8244
Other Eukaryotes     47198
Virus                 9800
Name: group, dtype: int64


In [64]:
contigs_out.to_csv("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/all_contigs_df.tsv", sep="\t", index=False)

In [53]:
# Summary table to combine reads to all contigs from the same sample-taxon pair
summary_nonviral = get_summary_table(contigs_out[contigs_out["group"]!="Virus"], 
                            colnames=["ska_genus","ska_species","collected_by","sample","taxid","group","family","sci_name"],
                                     metric=["read_count", "read_prop"])
summary_viral = get_summary_table(contigs_out[contigs_out["group"]=="Virus"], 
                            colnames=["ska_genus","ska_species","collected_by","sample","taxid","group","family","sci_name",
                                      "poly_group","baltimore_group","genome_description","number_of_segments_conventional"],
                            metric=["read_count", "read_prop"])
contigs_out_summary = pd.concat([summary_nonviral, summary_viral], sort=False, ignore_index = True)
# Rename column for consistency
contigs_out_summary.rename(columns={"read_count":"reads"}, inplace = True)
# Check all reads are accounted for in the summarizing
contigs_out['read_count'].sum() == contigs_out_summary['reads'].sum() #Shld be True

False

In [58]:
# Write to file
contigs_out_summary.to_csv("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/all_contigs_df_new.tsv", sep="\t", index=False)

In [244]:
# Write contig names to file for eukaryotes-in-bloodmeals analysis
category_names = ['Apicomplexa', "Schistosomatidae", "Onchocercidae",'Trypanosomatidae']
for catname in category_names:
    df = contigs_out[(contigs_out['family']==catname) & contigs_out['sample']]
    out = df['sample'] + "~" + df['contig_name']
    filename = "contig_names_" + catname + ".txt"
    print(catname + ": " + str(len(out)) + " contigs")
    out.to_csv("/Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/" + filename, sep="\n", index=False)

Apicomplexa: 474 contigs
Schistosomatidae: 0 contigs
Onchocercidae: 0 contigs
Trypanosomatidae: 788 contigs


For column: ska_genus
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: ska_species
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: collected_by
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: sample
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: taxid
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: group
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: family
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: sci_name
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: reads
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: read_prop
old df uniquely contains: {0.0031107467110445, 0.0005141869621923, 0.0003370765486892, 5.477349337468608e-05, 0.0003459322313546, 0.0001954177682535, 0.002

In [252]:
compare_new_to_old(5654,'Trypanosomatidae')


,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAGOAT002_S49_656110,0.000002,0.000002,0.000002,0.000002
AIAMAPIG001_S63_657713,0.000234,0.000234,0.000234,0.000234
AIAMAR001_LIV_S21_662106,0.000028,0.000028,0.000028,0.000028
AIAMAR002_LIV_S22_662107,0.000008,0.000008,0.000008,0.000008
AIAMAR004_LIV_S24_1_662161,0.000029,0.000029,0.000029,0.000029
...,...,...,...,...
SEZONOWIFRR071_LUNG_S62_673527,0.000048,0.000048,0.000048,0.000048
SEZONOWIFRR072_LUNG_S63_673528,0.000003,0.000003,0.000003,0.000003
SEZONOWOGDOG016_S23_657752,0.000001,0.000001,0.000001,0.000001


In [254]:
compare_new_to_old(5794,'Apicomplexa')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMADOG001_S40_657732,0.000006,0.000006,0.000006,0.000006
AIAMADOG002_OS_S64_673467,0.000002,0.000002,0.000002,0.000002
AIAMAGOAT001_S48_656109,0.000014,0.000014,0.000014,0.000014
AIAMAGOAT002_S49_656110,0.000037,0.000037,0.000037,0.000037
AIAMAGOAT003_S50_657720,0.000004,0.000004,0.000004,0.000004
...,...,...,...,...
SEZONOWOGDOG00_S2_657745,0.000069,0.000069,0.000069,0.000069
SEZONOWOGGOAT015_S28_657725,0.000229,0.000229,0.000229,0.000229
SEZONOWOGGOAT018_S66_1_657730,0.000080,0.000080,0.000080,0.000080


In [255]:
compare_new_to_old(6231,'Nematoda')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAAGR001_LIV_S46_662105,0.000009,0.000009,0.000009,0.000009
AIAMACAT001_S69_657769,0.008262,0.008262,0.008262,0.008262
AIAMADOG001_S40_657732,0.004408,0.004408,0.004408,0.004408
AIAMADOG002_OS_S64_673467,0.015888,0.015888,0.015888,0.015888
AIAMAGOAT001_S48_656109,0.000053,0.000053,0.000053,0.000053
...,...,...,...,...
SEZONOWOGGOAT018_S66_1_657730,0.000004,0.000004,0.000004,0.000004
SEZONOWOGR001_LIV_S47_666398,0.009960,0.009960,0.009960,0.009960
SEZONOWOGR002_LIV_S48_666373,0.011657,0.011657,0.011657,0.011657


In [256]:
compare_new_to_old(629,'Onchocercidae')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAGOAT003_S50_657720,3.828799e-07,NaN,NaN,NaN
AIAMAPIG002_S64_657714,2.479061e-06,NaN,NaN,NaN
AIAMAR002_LIV_S22_662107,1.338300e-05,NaN,NaN,NaN
AIAMAR005_LIV_S25_1_662162,1.974668e-06,NaN,NaN,NaN
AIAMA_GOT005_OS_S43_673459,2.314774e-05,NaN,NaN,NaN
...,...,...,...,...
SEZONOWOGDOG010_S3_657749,4.152486e-07,NaN,NaN,NaN
SEZONOWOGDOG014_S21_657751,1.040949e-06,NaN,NaN,NaN
SEZONOWOGDOG016_S23_657752,4.679272e-04,NaN,NaN,NaN


In [ ]:
compare_new_to_old(6029,'Microsporidia')

In [257]:
contigs_out[contigs_out.taxon_group == "Viruses"].groupby("family")["family"].count()

Series([], Name: family, dtype: int64)

In [258]:
contigs_out[contigs_out.taxon_group != "Viruses"].groupby("family")["family"].count()

family
Apicomplexa            474
Bacteria            478447
Microsporidia           31
Nematoda              4081
Other eukaryotes     45905
Other metazoa         4163
Trypanosomatidae       788
Name: family, dtype: int64

In [259]:
contigs_out[contigs_out.taxon_group != "Viruses"].groupby("group")["group"].count()

group
Bacteria            478447
Metazoa               8244
Other Eukaryotes     47198
Name: group, dtype: int64

array(['Bacteria', 'Other metazoa', 'Nematoda', 'Other eukaryotes',
       'Apicomplexa', 'Microsporidia', 'Trypanosomatidae', nan],
      dtype=object)

In [81]:
## WOLBACHIA
Bacteria_taxid = 2
Bacteria_contigs = get_rows_taxid(contig_stats_nonviral, taxid=Bacteria_taxid)

## METAZOANS
metazoan_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Metazoa"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
metazoan_contigs = group_at_higher_tax(
    metazoan_contigs, 
    taxonomic_group=["Nematoda"], 
    family_name="Other metazoa", 
    taxid_colname="taxid", 
    family_colname="family"
)

## METAZOANS
Bacteria_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Bacteria"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
Bacteria_contigs = group_at_higher_tax(
    Bacteria_contigs, 
    taxonomic_group=["Bacteria"], 
    family_name="Other Bacteria", 
    taxid_colname="taxid", 
    family_colname="family"
)
## CHORDATA
# Option 1: Using numerical taxid
CHORDATA_TAXID = 7711
chordate_contigs = get_rows_taxid(contig_stats_nonviral, taxid=CHORDATA_TAXID)

# Option 2: Filtering by 'phylum'
chordate_contigs = contig_stats_nonviral[contig_stats_nonviral["phylum"] == "Chordata"]

# Proceed with grouping if necessary
chordate_contigs = group_at_higher_tax(
    chordate_contigs, 
    taxonomic_group=["Pecora", "Aves", "Carnivora", "Rodentia", "Leporidae", "Apicomplexa","Microsporidia", "Platyhelminthes"], 
    family_name="Other chordates", 
    taxid_colname="taxid", 
    family_colname="phylum"  # Ensure this is the correct column
)

## OTHER EUKARYOTES
eukaryotic_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Eukaryota"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
eukaryotic_contigs = group_at_higher_tax(
    eukaryotic_contigs, 
    taxonomic_group=["Trypanosomatidae", "Schistosomatidae", "Onchocercidae", ], 
    family_name="Other eukaryotes", 
    taxid_colname="taxid", 
    family_colname="family"
)


In [77]:
contig_stats_nonviral.to_csv(gitdir + "Non_virus_groups.csv")

False

In [193]:
import pandas as pd

# Set your file paths
input_file = (gitdir + "Non_virus_groups.csv")  # Input CSV file
output_file = (gitdir + "chordata_tax_id_sci_name_families.csv")  # Output CSV file

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Filter rows where the phylum column is "Chordata"
chordata_df = df[df['phylum'] == 'Chordata']

# Select unique combinations of tax_id, sci_name, and family
unique_data = chordata_df[['tax_id', 'sci_name', 'family']].drop_duplicates()

# Save the unique data to a new CSV file
unique_data.to_csv(output_file, index=False)

print(f"Unique data (tax_id, sci_name, family) saved to {output_file}.")

Unique data (tax_id, sci_name, family) saved to /Volumes/aine_store/SENZOR_project/SPlited_SENSOR_porject/IDseq/Cluster_contigs/chordata_tax_id_sci_name_families.csv.


In [203]:
## WOLBACHIA
Bacteria_taxid = 952
Bacteria_contigs = get_rows_taxid(contig_stats_nonviral, taxid=Arthropod_taxid)

## METAZOANS
metazoan_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Metazoa"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
metazoan_contigs = group_at_higher_tax(
    metazoan_contigs, 
    taxonomic_group=["Nematoda"], 
    family_name="Other metazoa", 
    taxid_colname="taxid", 
    family_colname="family"
)

## METAZOANS
Bacteria_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Bacteria"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
Bacteria_contigs = group_at_higher_tax(
    Bacteria_contigs, 
    taxonomic_group=["Bacteria"], 
    family_name="Other Bacteria", 
    taxid_colname="taxid", 
    family_colname="family"
)
## CHORDATA
# Option 1: Using numerical taxid
CHORDATA_TAXID = 7711
chordate_contigs = get_rows_taxid(contig_stats_nonviral, taxid=CHORDATA_TAXID)

# Option 2: Filtering by 'phylum'
chordate_contigs = contig_stats_nonviral[contig_stats_nonviral["phylum"] == "Chordata"]

# Proceed with grouping if necessary
chordate_contigs = group_at_higher_tax(
    chordate_contigs, 
    taxonomic_group=["Pecora", "Aves", "Carnivora", "Rodentia", "Leporidae", "Apicomplexa","Microsporidia", "Platyhelminthes", "Fish"], 
    family_name="Other chordates", 
    taxid_colname="taxid", 
    family_colname="phylum"  # Ensure this is the correct column
)

## OTHER EUKARYOTES
eukaryotic_contigs = contig_stats_nonviral[contig_stats_nonviral["taxon_group"] == "Eukaryota"]
# In the 'family' column, add information about the higher taxonomic grouping for taxids
eukaryotic_contigs = group_at_higher_tax(
    eukaryotic_contigs, 
    taxonomic_group=["Trypanosomatidae", "Schistosomatidae", "Onchocercidae", ], 
    family_name="Other eukaryotes", 
    taxid_colname="taxid", 
    family_colname="family"
)


/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 85948 was translated into 147567
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 2991674 was translated into 3383038
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 7604 was translated into 133437
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 2052317 was translated into 3107663
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Users/ifeanyi/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiqu

group
Bacteria            478447
Metazoa               8244
Other Eukaryotes     47198
Virus                 9800
Name: group, dtype: int64


In [ ]:
import pandas as pd
from ete3 import NCBITaxa

# Initialize NCBI Taxonomy
ncbi = NCBITaxa()

# Combine nonviral contigs with correct group and family assignments
contigs_nonviral = pd.concat([
    wolbachia_contigs.assign(group="Wolbachia").assign(family="Wolbachia"), 
    metazoan_contigs.assign(group="Metazoa"),
    chordate_contigs.assign(group="Chordates"),
    Bacteria_contigs.assign(group="Bacteria"),
    eukaryotic_contigs.assign(group="Other Eukaryotes")
], sort=False)

# Ensure 'taxid' is integer, handling missing values appropriately
contigs_nonviral['taxid'] = contigs_nonviral['taxid'].fillna(0).astype(int)

# Extract unique taxids, excluding placeholder like 0
unique_taxids = contigs_nonviral['taxid'].unique()
unique_taxids = unique_taxids[unique_taxids != 0]

# Fetch taxid to scientific name mapping
translator = ncbi.get_taxid_translator(unique_taxids.tolist())

# Identify missing taxids
missing_taxids = set(unique_taxids) - set(translator.keys())
#print(f"Missing Taxids: {missing_taxids}")  # Optional

# Create complete translator with 'Unknown' for missing taxids
translator_complete = {taxid: translator.get(taxid, "Unknown") for taxid in unique_taxids}

# Map 'taxid' to 'sci_name'
contigs_nonviral['sci_name'] = contigs_nonviral['taxid'].map(translator_complete)

# Handle placeholder taxids if necessary
contigs_nonviral.loc[contigs_nonviral['taxid'] == 0, 'sci_name'] = "Unknown"

# Replace 'sci_name' for specific groups
contigs_nonviral.loc[contigs_nonviral['group'] == 'Platyhelminthes', 'sci_name'] = 'Platyhelminthes'
contigs_nonviral.loc[contigs_nonviral['group'] == 'Wolbachia', 'sci_name'] = 'Wolbachieae'

# Handle FutureWarning by casting boolean columns explicitly
bool_cols = contigs_nonviral.select_dtypes(include=['bool']).columns
contigs_nonviral[bool_cols] = contigs_nonviral[bool_cols].astype(bool)

# Display the resulting DataFrame
#print(contigs_nonviral.head())


In [236]:
contigs_out = pd.concat([contigs_nonviral, contigs_viral], sort=False)
# Count total number of contigs in each broad group
numbers = contigs_out.groupby("group")["group"].count()
print(numbers)

group
Bacteria            478447
Metazoa               8244
Other Eukaryotes     47198
Virus                 9800
Name: group, dtype: int64


In [380]:
contigs_out=pd.read_csv(gitdir + "contigs_out.csv")

/var/folders/xt/vz8fznds4hdbfcxyds95mtp00000gp/T/ipykernel_13831/269359503.py:1: DtypeWarning: Columns (5,16,19,24,25,26,31,32,33,42,43,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  contigs_out=pd.read_csv(gitdir + "contigs_out.csv")


In [209]:
# Summary table to combine reads to all contigs from the same sample-taxon pair
summary_nonviral = get_summary_table(contigs_out[contigs_out["group"]!="Virus"], 
                            colnames=["ska_genus","ska_species","collected_by","sample","taxid","group","family","sci_name"],
                                     metric=["read_count", "read_prop"])
summary_viral = get_summary_table(contigs_out[contigs_out["group"]=="Virus"], 
                            colnames=["ska_genus","ska_species","collected_by","sample","taxid","group","family","sci_name",
                                      "poly_group","baltimore_group","genome_description","number_of_segments_conventional"],
                            metric=["read_count", "read_prop"])
contigs_out_summary = pd.concat([summary_nonviral, summary_viral], sort=False, ignore_index = True)
# Rename column for consistency
contigs_out_summary.rename(columns={"read_count":"reads"}, inplace = True)
# Check all reads are accounted for in the summarizing
contigs_out['read_count'].sum() == contigs_out_summary['reads'].sum() #Shld be True

False

In [211]:
summary_nonviral

,ska_genus,ska_species,collected_by,sample,taxid,group,family,sci_name,read_count,read_prop
88375,Swine,Pig,Ebonyi,AIAMA_PIG003_RS_S51_673463,3242775.0,Bacteria,Bacteria,Bacteria,8,6.223202e-07
88374,Swine,Pig,Ebonyi,AIAMA_PIG003_RS_S51_673463,3113737.0,Bacteria,Bacteria,Bacteria,8,6.223202e-07
88373,Swine,Pig,Ebonyi,AIAMA_PIG003_RS_S51_673463,3111978.0,Bacteria,Bacteria,Bacteria,10,7.779002e-07
88372,Swine,Pig,Ebonyi,AIAMA_PIG003_RS_S51_673463,3075544.0,Bacteria,Bacteria,Bacteria,8,6.223202e-07
88371,Swine,Pig,Ebonyi,AIAMA_PIG003_RS_S51_673463,3050609.0,Bacteria,Bacteria,Bacteria,155,1.205745e-05
...,...,...,...,...,...,...,...,...,...,...
1364,Avian,Chicken,Ebonyi,AINWZCHICKEN002_S57_655748,197.0,Bacteria,Bacteria,Bacteria,24,6.495805e-06
2,Avian,Chicken,Ebonyi,AINWZCHICKEN001_S56_655747,197.0,Bacteria,Bacteria,Bacteria,14,9.222245e-07
1,Avian,Chicken,Ebonyi,AINWZCHICKEN001_S56_655747,195.0,Bacteria,Bacteria,Bacteria,22,1.449210e-06
1363,Avian,Chicken,Ebonyi,AINWZCHICKEN002_S57_655748,192.0,Bacteria,Bacteria,Bacteria,46,1.245029e-05


In [389]:
# Write to file
contigs_out_summary.to_csv(gitdir + "all_contigs_df_new.tsv", sep="\t", index=False)

In [413]:
def whatsin(df):
    return [{x:df[x].unique()} for x in df.columns.tolist()]

def get_desc_contigs(contig_df, txid):
    taxa = ncbi.get_descendant_taxa(txid, intermediate_nodes=True)
    taxa.append(txid)
    df = contig_df[contig_df["taxid"].apply(lambda x: x in taxa)]
    return df

In [213]:
# Checking column names and values in old vs. new
old_df = pd.read_csv(gitdir + "all_contigs_df_new.tsv", sep="\t", header=0)
for mycol in old_df.columns:
    print("For column: " + str(mycol))
    a = set(old_df[mycol].unique())
    b = set(contigs_out_summary[mycol].unique())
    print("old df uniquely contains: " + str(a-b))
    print("new df uniquely contains: " + str(b-a))
    print("------")

For column: ska_genus
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: ska_species
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: collected_by
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: sample
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: taxid
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: group
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: family
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: sci_name
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: reads
old df uniquely contains: set()
new df uniquely contains: set()
------
For column: read_prop
old df uniquely contains: {0.0031107467110445, 0.0005141869621923, 0.0003370765486892, 5.477349337468608e-05, 0.0003459322313546, 0.0001954177682535, 0.002

In [215]:
txid = 1068625
category_name = 'Trypanosomatidae'
new_level = 'family'
old_level = 'family'

def compare_new_to_old(txid, category_name, new_level = 'family', old_level = 'family'):
    df = pd.concat([ get_desc_contigs(contigs_out,txid).groupby('sample')['read_prop'].sum().rename('new_contigs_by_taxid'),
         contigs_out[(contigs_out[new_level]==category_name)].groupby('sample')['read_prop'].sum().rename('new_contigs_by_category'),
         contigs_out_summary[contigs_out_summary[new_level]==category_name].groupby('sample')['read_prop'].sum().rename('new_summary_by_category'),
         old_df[old_df[old_level]==category_name].groupby('sample')['read_prop'].sum().rename('old_by_category') ],
         axis=1, sort=False)
    return df

In [217]:
compare_new_to_old(1068625,'Trypanosomatidae')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAGOAT002_S49_656110,9.538251e-07,0.000002,0.000002,0.000002
AIAMAR001_LIV_S21_662106,2.790899e-05,0.000028,0.000028,0.000028
AIAMAR002_LIV_S22_662107,8.364374e-06,0.000008,0.000008,0.000008
AIAMAR004_LIV_S24_1_662161,2.852065e-05,0.000029,0.000029,0.000029
AIAMAR005_LIV_S25_1_662162,3.653136e-05,0.000037,0.000037,0.000037
...,...,...,...,...
OD_ZAD_SQL028_LUNG_S18_673624,NaN,0.000004,0.000004,0.000004
SENONOWISUGOT001_OS_S80_673658,NaN,0.000018,0.000018,0.000018
SENONOWISUGOT005_RS_S71_673662,NaN,0.000027,0.000027,0.000027


In [219]:
compare_new_to_old(6231,'Nematoda')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAAGR001_LIV_S46_662105,0.000009,0.000009,0.000009,0.000009
AIAMACAT001_S69_657769,0.008262,0.008262,0.008262,0.008262
AIAMADOG001_S40_657732,0.004408,0.004408,0.004408,0.004408
AIAMADOG002_OS_S64_673467,0.015888,0.015888,0.015888,0.015888
AIAMAGOAT001_S48_656109,0.000053,0.000053,0.000053,0.000053
...,...,...,...,...
SEZONOWOGGOAT018_S66_1_657730,0.000004,0.000004,0.000004,0.000004
SEZONOWOGR001_LIV_S47_666398,0.009960,0.009960,0.009960,0.009960
SEZONOWOGR002_LIV_S48_666373,0.011657,0.011657,0.011657,0.011657


In [220]:
compare_new_to_old(629,'Onchocercidae')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAGOAT003_S50_657720,3.828799e-07,NaN,NaN,NaN
AIAMAPIG002_S64_657714,2.479061e-06,NaN,NaN,NaN
AIAMAR002_LIV_S22_662107,1.338300e-05,NaN,NaN,NaN
AIAMAR005_LIV_S25_1_662162,1.974668e-06,NaN,NaN,NaN
AIAMA_GOT005_OS_S43_673459,2.314774e-05,NaN,NaN,NaN
...,...,...,...,...
SEZONOWOGDOG010_S3_657749,4.152486e-07,NaN,NaN,NaN
SEZONOWOGDOG014_S21_657751,1.040949e-06,NaN,NaN,NaN
SEZONOWOGDOG016_S23_657752,4.679272e-04,NaN,NaN,NaN


In [221]:
compare_new_to_old(5794,'Apicomplexa')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMADOG001_S40_657732,0.000006,0.000006,0.000006,0.000006
AIAMADOG002_OS_S64_673467,0.000002,0.000002,0.000002,0.000002
AIAMAGOAT001_S48_656109,0.000014,0.000014,0.000014,0.000014
AIAMAGOAT002_S49_656110,0.000037,0.000037,0.000037,0.000037
AIAMAGOAT003_S50_657720,0.000004,0.000004,0.000004,0.000004
...,...,...,...,...
SEZONOWOGDOG00_S2_657745,0.000069,0.000069,0.000069,0.000069
SEZONOWOGGOAT015_S28_657725,0.000229,0.000229,0.000229,0.000229
SEZONOWOGGOAT018_S66_1_657730,0.000080,0.000080,0.000080,0.000080


In [222]:
compare_new_to_old(6029,'Microsporidia')

,new_contigs_by_taxid,new_contigs_by_category,new_summary_by_category,old_by_category
sample,,,,
AIAMAGOAT001_S48_656109,2.848045e-07,2.848045e-07,2.848045e-07,2.848045e-07
AIAMAPIG001_S63_657713,1.387435e-05,1.387435e-05,1.387435e-05,1.387435e-05
AIAMAPIG002_S64_657714,1.534212e-05,1.534212e-05,1.534212e-05,1.534212e-05
AIAMASHEEP001_S53_656078,2.449067e-06,2.449067e-06,2.449067e-06,2.449067e-06
AIAMA_SHP004_OS_S46_673464,2.392719e-07,2.392719e-07,2.392719e-07,2.392719e-07
AIAMA_SHP006_OS_S48_673466,1.925876e-05,1.925876e-05,1.925876e-05,1.925876e-05
AIMASHEEP003_S55_656080,1.173220e-05,1.173220e-05,1.173220e-05,1.173220e-05
AIMGLDOG001_S59_657733,8.058039e-05,8.058039e-05,8.058039e-05,8.058039e-05
AIMGL_GOT003_OS_S65_673468,2.310064e-05,2.310064e-05,2.310064e-05,2.310064e-05


In [223]:
contigs_out[contigs_out.taxon_group == "Viruses"].groupby("family")["family"].count()


Series([], Name: family, dtype: int64)

In [224]:
contigs_out[contigs_out.taxon_group != "Viruses"].groupby("family")["family"].count()

family
Apicomplexa            474
Bacteria            478447
Microsporidia           31
Nematoda              4081
Other eukaryotes     45905
Other metazoa         4163
Trypanosomatidae       788
Name: family, dtype: int64

In [225]:
contigs_out[contigs_out.taxon_group != "Viruses"].groupby("group")["group"].count()

group
Bacteria            478447
Metazoa               8244
Other Eukaryotes     47198
Name: group, dtype: int64

In [226]:
contigs_out.family.unique()

array(['Bacteria', 'Other metazoa', 'Nematoda', 'Other eukaryotes',
       'Apicomplexa', 'Microsporidia', 'Trypanosomatidae', nan],
      dtype=object)